<a href="https://colab.research.google.com/github/cbachen1997/CAUstudy/blob/master/ZB_semitriplet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')
####google云盘授权#####
##每个notebook执行一次###
__author__='CBA'
from google.colab import drive

#增加PyDrive操作库
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#授权登录
auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()
drive=GoogleDrive(gauth)

Mounted at /content/gdrive


In [ ]:
from __future__ import print_function, division
import tensorflow as tf
from sklearn.utils import shuffle
import keras as K
import keras.layers as L
import numpy as np
import os
import time
import h5py
import argparse 
import random
import cv2
from tqdm import *
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import LambdaCallback
# from keras.callbacks import TensorBoard
from keras import regularizers
from tensorflow.keras.models import Model
from __future__ import print_function, division
from keras.layers import *
# from keras.layers import Dense,Dropout
from sklearn.utils import shuffle
from keras.layers.core import Flatten
from keras.utils import to_categorical
# from tensorflow.keras.layers import Conv2D
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
# from keras.layers.core import Lambda
from keras import backend as Kb
from collections import Counter


In [ ]:
#全局参数
inputshape=(11,11,12)
num=10#每类抽10个
alpha=0.05
#标签样本数据npy
scale=1000#每次抽取的无标签数量构建伪样本池
topk=25#ranking前k个


In [ ]:
from scipy.ndimage.interpolation import map_coordinates as sp_map_coordinates
def tf_flatten(a):
    """Flatten tensor"""
    return tf.reshape(a, [-1])


def tf_repeat(a, repeats, axis=0):
    """TensorFlow version of np.repeat for 1D"""
    # https://github.com/tensorflow/tensorflow/issues/8521
    assert len(a.get_shape()) == 1

    a = tf.expand_dims(a, -1)
    a = tf.tile(a, [1, repeats])
    a = tf_flatten(a)
    return a


def tf_repeat_2d(a, repeats):
    """Tensorflow version of np.repeat for 2D"""

    assert len(a.get_shape()) == 2
    a = tf.expand_dims(a, 0)
    a = tf.tile(a, [repeats, 1, 1])
    return a


def tf_map_coordinates(input, coords, order=1):
    """Tensorflow verion of scipy.ndimage.map_coordinates

    Note that coords is transposed and only 2D is supported

    Parameters
    ----------
    input : tf.Tensor. shape = (s, s)
    coords : tf.Tensor. shape = (n_points, 2)
    """

    assert order == 1

    coords_lt = tf.cast(tf.floor(coords), 'int32')
    #coords_rb = tf.cast(tf.ceil(coords), 'int32')
    coords_rb = tf.cast(tf.math.ceil(coords), 'int32')
    coords_lb = tf.stack([coords_lt[:, 0], coords_rb[:, 1]], axis=1)
    coords_rt = tf.stack([coords_rb[:, 0], coords_lt[:, 1]], axis=1)

    vals_lt = tf.gather_nd(input, coords_lt)
    vals_rb = tf.gather_nd(input, coords_rb)
    vals_lb = tf.gather_nd(input, coords_lb)
    vals_rt = tf.gather_nd(input, coords_rt)

    coords_offset_lt = coords - tf.cast(coords_lt, 'float32')
    vals_t = vals_lt + (vals_rt - vals_lt) * coords_offset_lt[:, 0]
    vals_b = vals_lb + (vals_rb - vals_lb) * coords_offset_lt[:, 0]
    mapped_vals = vals_t + (vals_b - vals_t) * coords_offset_lt[:, 1]

    return mapped_vals


def sp_batch_map_coordinates(inputs, coords):
    """Reference implementation for batch_map_coordinates"""
    coords = coords.clip(0, inputs.shape[1] - 1)
    mapped_vals = np.array([
        sp_map_coordinates(input, coord.T, mode='nearest', order=1)
        for input, coord in zip(inputs, coords)
    ])
    return mapped_vals


def tf_batch_map_coordinates(input, coords, order=1):
    """Batch version of tf_map_coordinates

    Only supports 2D feature maps

    Parameters
    ----------
    input : tf.Tensor. shape = (b, s, s)
    coords : tf.Tensor. shape = (b, n_points, 2)

    Returns
    -------
    tf.Tensor. shape = (b, s, s)
    """

    input_shape = tf.shape(input)
    batch_size = input_shape[0]
    input_size = input_shape[1]
    n_coords = tf.shape(coords)[1]

    coords = tf.clip_by_value(coords, 0, tf.cast(input_size, 'float32') - 1)
    coords_lt = tf.cast(tf.floor(coords), 'int32')
    #coords_rb = tf.cast(tf.ceil(coords), 'int32')
    coords_rb = tf.cast(tf.math.ceil(coords), 'int32')
    coords_lb = tf.stack([coords_lt[..., 0], coords_rb[..., 1]], axis=-1)
    coords_rt = tf.stack([coords_rb[..., 0], coords_lt[..., 1]], axis=-1)

    idx = tf_repeat(tf.range(batch_size), n_coords)

    def _get_vals_by_coords(input, coords):
        indices = tf.stack([
            idx, tf_flatten(coords[..., 0]), tf_flatten(coords[..., 1])
        ], axis=-1)
        vals = tf.gather_nd(input, indices)
        vals = tf.reshape(vals, (batch_size, n_coords))
        return vals

    vals_lt = _get_vals_by_coords(input, coords_lt)
    vals_rb = _get_vals_by_coords(input, coords_rb)
    vals_lb = _get_vals_by_coords(input, coords_lb)
    vals_rt = _get_vals_by_coords(input, coords_rt)

    coords_offset_lt = coords - tf.cast(coords_lt, 'float32')
    vals_t = vals_lt + (vals_rt - vals_lt) * coords_offset_lt[..., 0]
    vals_b = vals_lb + (vals_rb - vals_lb) * coords_offset_lt[..., 0]
    mapped_vals = vals_t + (vals_b - vals_t) * coords_offset_lt[..., 1]

    return mapped_vals


def sp_batch_map_offsets(input, offsets):
    """Reference implementation for tf_batch_map_offsets"""

    batch_size = input.shape[0]
    input_size = input.shape[1]

    offsets = offsets.reshape(batch_size, -1, 2)
    grid = np.stack(np.mgrid[:input_size, :input_size], -1).reshape(-1, 2)
    grid = np.repeat([grid], batch_size, axis=0)
    coords = offsets + grid
    coords = coords.clip(0, input_size - 1)

    mapped_vals = sp_batch_map_coordinates(input, coords)
    return mapped_vals


def tf_batch_map_offsets(input, offsets, order=1):
    """Batch map offsets into input

    Parameters
    ---------
    input : tf.Tensor. shape = (b, s, s)
    offsets: tf.Tensor. shape = (b, s, s, 2)

    Returns
    -------
    tf.Tensor. shape = (b, s, s)
    """

    input_shape = tf.shape(input)
    batch_size = input_shape[0]
    input_size = input_shape[1]

    offsets = tf.reshape(offsets, (batch_size, -1, 2))
    grid = tf.meshgrid(
        tf.range(input_size), tf.range(input_size), indexing='ij'
    )
    grid = tf.stack(grid, axis=-1)
    grid = tf.cast(grid, 'float32')
    grid = tf.reshape(grid, (-1, 2))
    grid = tf_repeat_2d(grid, batch_size)
    coords = offsets + grid

    mapped_vals = tf_batch_map_coordinates(input, coords)
    return mapped_vals

In [ ]:
class ConvOffset2D(Conv2D):
    """ConvOffset2D

    Convolutional layer responsible for learning the 2D offsets and output the
    deformed feature map using bilinear interpolation

    Note that this layer does not perform convolution on the deformed feature
    map. See get_deform_cnn in cnn.py for usage
    """

    def __init__(self, filters, init_normal_stddev=0.01, **kwargs):
        """Init

        Parameters
        ----------
        filters : int
            Number of channel of the input feature map
        init_normal_stddev : float
            Normal kernel initialization
        **kwargs:
            Pass to superclass. See Con2D layer in Keras
        """

        self.filters = filters
        super(ConvOffset2D, self).__init__(
            self.filters * 2, (3, 3), padding='same', use_bias=False,
            kernel_initializer=K.initializers.RandomNormal(0, init_normal_stddev),
            **kwargs
        )

    def call(self, x):
        """Return the deformed featured map"""
        x_shape = x.get_shape()
        offsets = super(ConvOffset2D, self).call(x)

        # offsets: (b*c, h, w, 2)
        offsets = self._to_bc_h_w_2(offsets, x_shape)

        # x: (b*c, h, w)
        x = self._to_bc_h_w(x, x_shape)

        # X_offset: (b*c, h, w)
        x_offset = tf_batch_map_offsets(x, offsets)

        # x_offset: (b, h, w, c)
        x_offset = self._to_b_h_w_c(x_offset, x_shape)

        return x_offset

    def compute_output_shape(self, input_shape):
        """Output shape is the same as input shape

        Because this layer does only the deformation part
        """
        return input_shape

    @staticmethod
    def _to_bc_h_w_2(x, x_shape):
        """(b, h, w, 2c) -> (b*c, h, w, 2)"""
        x = tf.transpose(x, [0, 3, 1, 2])
        x = tf.reshape(x, (-1, int(x_shape[1]), int(x_shape[2]), 2))
        return x

    @staticmethod
    def _to_bc_h_w(x, x_shape):
        """(b, h, w, c) -> (b*c, h, w)"""
        x = tf.transpose(x, [0, 3, 1, 2])
        x = tf.reshape(x, (-1, int(x_shape[1]), int(x_shape[2])))
        return x

    @staticmethod
    def _to_b_h_w_c(x, x_shape):
        """(b*c, h, w) -> (b, h, w, c)"""
        x = tf.reshape(
            x, (-1, int(x_shape[3]), int(x_shape[1]), int(x_shape[2]))
        )
        x = tf.transpose(x, [0, 2, 3, 1])
        return x
def BN_LeakyReLU(input):
    
    norm = L.BatchNormalization(axis=-1)(input)
    output = L.advanced_activations.LeakyReLU(alpha=0.2)(norm)
    
    return output

In [ ]:
######################################backbone部分######################################
def feature_extraction_CNN(input_shape, n_filters=64):
    # X_input=L.Input(input_shape)
    conv1 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(input_shape)
   
    conv1 = BN_LeakyReLU(conv1)
    conv1 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(conv1)
    conv1 = BN_LeakyReLU(conv1)
    
    pool1 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv1)
    # print(pool1)
    #可变形block A1
   
    offset_conv2_1 = ConvOffset2D(2*n_filters)(pool1)
   
    conv2_1 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_1)
    conv2_1 = BN_LeakyReLU(conv2_1)
    
    offset_conv2_2 = ConvOffset2D(n_filters)(conv2_1)
    conv2_2 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_2)
    conv2_2 = BN_LeakyReLU(conv2_2)
    
    offset_conv2_3 = ConvOffset2D(int(0.5*n_filters))(conv2_2)
    conv2_3 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_3)
    conv2_3 = BN_LeakyReLU(conv2_3)
    
    conv2_4 = L.concatenate([conv2_1, conv2_2, conv2_3], axis=-1)
    # print(conv2_4)
    conv2_5 = L.merge.add([conv2_4, pool1])

    #可变形block A2
    offset_conv2_6 = ConvOffset2D(2*n_filters)(conv2_5)
    conv2_6 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_6)
    conv2_6 = BN_LeakyReLU(conv2_6)
    
    offset_conv2_7 = ConvOffset2D(n_filters)(conv2_6)
    conv2_7 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_7)
    conv2_7 = BN_LeakyReLU(conv2_7)
    
    offset_conv2_8 = ConvOffset2D(int(0.5*n_filters))(conv2_7)
    conv2_8 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_8)
    conv2_8 = BN_LeakyReLU(conv2_8)
    
    conv2_9 = L.concatenate([conv2_6, conv2_7, conv2_8], axis=-1)
    
    conv2_10 = L.merge.add([conv2_9, conv2_5])
    
    
    #pool2 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv2_5)
    
    conv3 = L.Conv2D(4*n_filters, (3, 3), padding='valid', strides=(2, 2), kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(conv2_5)
    
    offset_conv3_1 = ConvOffset2D(4*n_filters)(conv3)
    
    conv3_1 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_1)
    conv3_1 = BN_LeakyReLU(conv3_1)
    
    offset_conv3_2 = ConvOffset2D(2*n_filters)(conv3_1)
    conv3_2 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_2)
    conv3_2 = BN_LeakyReLU(conv3_2)
    offset_conv3_3 = ConvOffset2D(n_filters)(conv3_2)
    conv3_3 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_3)
    conv3_3 = BN_LeakyReLU(conv3_3)
    
    conv3_4 = L.concatenate([conv3_1, conv3_2, conv3_3], axis=-1)
    
    conv3_5 = L.merge.add([conv3_4, conv3])
 
    #改了一下4*
    offset_conv3_6 = ConvOffset2D(n_filters)(conv3_2)
    
    conv3_6 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_6)
    conv3_6 = BN_LeakyReLU(conv3_6)
    
    offset_conv3_7 = ConvOffset2D(2*n_filters)(conv3_6)
    conv3_7 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_7)
    conv3_7 = BN_LeakyReLU(conv3_7)
    
    offset_conv3_8 = ConvOffset2D(n_filters)(conv3_7)
    conv3_8 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_8)
    conv3_8 = BN_LeakyReLU(conv3_8)
    
    conv3_9 = L.concatenate([conv3_6, conv3_7, conv3_8], axis=-1)
    
    conv3_10 = L.merge.add([conv3_9, conv3_5])
    
    
    #pool3 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv3)
    #输出128维计算度量
    conv4 = L.Conv2D(n_filters*2, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(conv3_10)
    # print(conv4)
    gap = L.GlobalAvgPool2D()(conv4)
    # print(gap)
    #增加功能
    X = Dense(1024,kernel_regularizer=regularizers.l2(0.01),name='dense_layer1')(gap)
    X1 = Dropout(rate=0.5)(X)
    X2 = Dense(256,kernel_regularizer=regularizers.l2(0.01),name='dense_layer2')(X1)
    spatial_result = X2
    print('输出向量维度：' + str(spatial_result.shape))
    return spatial_result
def feature_extraction_CNN_tri(input_shape, n_filters=64):
    X_input=L.Input(input_shape)
    conv1 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(X_input)
   
    conv1 = BN_LeakyReLU(conv1)
    conv1 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(conv1)
    conv1 = BN_LeakyReLU(conv1)
    
    pool1 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv1)
    # print(pool1)
    #可变形block A1
   
    offset_conv2_1 = ConvOffset2D(2*n_filters)(pool1)
   
    conv2_1 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_1)
    conv2_1 = BN_LeakyReLU(conv2_1)
    
    offset_conv2_2 = ConvOffset2D(n_filters)(conv2_1)
    conv2_2 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_2)
    conv2_2 = BN_LeakyReLU(conv2_2)
    
    offset_conv2_3 = ConvOffset2D(int(0.5*n_filters))(conv2_2)
    conv2_3 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_3)
    conv2_3 = BN_LeakyReLU(conv2_3)
    
    conv2_4 = L.concatenate([conv2_1, conv2_2, conv2_3], axis=-1)
    # print(conv2_4)
    conv2_5 = L.merge.add([conv2_4, pool1])

    #可变形block A2
    offset_conv2_6 = ConvOffset2D(2*n_filters)(conv2_5)
    conv2_6 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_6)
    conv2_6 = BN_LeakyReLU(conv2_6)
    
    offset_conv2_7 = ConvOffset2D(n_filters)(conv2_6)
    conv2_7 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_7)
    conv2_7 = BN_LeakyReLU(conv2_7)
    
    offset_conv2_8 = ConvOffset2D(int(0.5*n_filters))(conv2_7)
    conv2_8 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_8)
    conv2_8 = BN_LeakyReLU(conv2_8)
    
    conv2_9 = L.concatenate([conv2_6, conv2_7, conv2_8], axis=-1)
    
    conv2_10 = L.merge.add([conv2_9, conv2_5])
    
    
    #pool2 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv2_5)
    
    conv3 = L.Conv2D(4*n_filters, (3, 3), padding='valid', strides=(2, 2), kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(conv2_5)
    
    offset_conv3_1 = ConvOffset2D(4*n_filters)(conv3)
    
    conv3_1 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_1)
    conv3_1 = BN_LeakyReLU(conv3_1)
    
    offset_conv3_2 = ConvOffset2D(2*n_filters)(conv3_1)
    conv3_2 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_2)
    conv3_2 = BN_LeakyReLU(conv3_2)
    offset_conv3_3 = ConvOffset2D(n_filters)(conv3_2)
    conv3_3 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_3)
    conv3_3 = BN_LeakyReLU(conv3_3)
    
    conv3_4 = L.concatenate([conv3_1, conv3_2, conv3_3], axis=-1)
    
    conv3_5 = L.merge.add([conv3_4, conv3])
 
    #改了一下4*
    offset_conv3_6 = ConvOffset2D(n_filters)(conv3_2)
    
    conv3_6 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_6)
    conv3_6 = BN_LeakyReLU(conv3_6)
    
    offset_conv3_7 = ConvOffset2D(2*n_filters)(conv3_6)
    conv3_7 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_7)
    conv3_7 = BN_LeakyReLU(conv3_7)
    
    offset_conv3_8 = ConvOffset2D(n_filters)(conv3_7)
    conv3_8 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_8)
    conv3_8 = BN_LeakyReLU(conv3_8)
    
    conv3_9 = L.concatenate([conv3_6, conv3_7, conv3_8], axis=-1)
    
    conv3_10 = L.merge.add([conv3_9, conv3_5])
    
    
    #pool3 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv3)
    #输出128维计算度量
    conv4 = L.Conv2D(n_filters*2, (3, 3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(conv3_10)
    # print(conv4)
    gap = L.GlobalAvgPool2D()(conv4)
    # print(gap)
    #增加功能
    X = Dense(1024,kernel_regularizer=regularizers.l2(0.01),name='dense_layer1')(gap)
    X1 = Dropout(rate=0.5)(X)
    X2 = Dense(256,kernel_regularizer=regularizers.l2(0.01),name='dense_layer2')(X1)
    spatial_result = X2
    print('输出向量维度：' + str(spatial_result.shape))
    model = Model(inputs=X_input,outputs=spatial_result,name='triplet_Model')
    return model
    # return spatial_result

In [ ]:
#1 抽取样本(每类num个)
def sampling(dataset_X,dataset_Y,num):
  X = []
  Y = []
  flag = 0
  for i in range(1,8):#7类
    classes = i
    # print('当前类别为：'+ str(classes))
    current_dataset = dataset_X[dataset_Y == classes]
    #随机抽
    np.random.shuffle(current_dataset)
    random_samples = current_dataset[0:num]
    if flag == 0 :
      random_samples = random_samples.tolist()
      X.append(random_samples)
      Y = Y + [classes for j in range(0,num)]
      X = np.asarray(X[0],dtype=float)
      flag = 1
    else:
      X = np.concatenate((X,random_samples),axis=0)
      Y = Y + [classes for j in range(0,num)]
  print('当前采样集大小:'+str(X.shape))
  Y = np.asarray(Y ,dtype = int)
  return X,Y#Y没啥用
def sampling_unlabel(unlabeled_X,scale):
  # np.random.shuffle(unlabeled_X)
  pe = np.random.permutation(unlabeled_X.shape[0])
  # print(pe)
  unlabeled_X =unlabeled_X[pe]
  unlabeled_X = unlabeled_X[0:scale,:]
  # print(unlabeled_X.shape)
  return unlabeled_X
#2 embedding
# def embedding(datanpy,model):
#   length  = datanpy.shape[0]
#   embed_array = np.zeros((datanpy.shape[0],256))
#   for i in range(length):
#     embedding=model.predict_on_batch(datanpy)
#   return embedding
#3 构建困难三元组
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import pdist
def get_hard_triplet(samples_X,samples_Y,model):
  """
  samples:采样结果
  model:模型
  """
  num_ = samples_X.shape[0]
  embed = model.predict(samples_X)
  hard_triplet=np.zeros((num_,3,samples_X.shape[1],samples_X.shape[2],samples_X.shape[3]))
  count = 0 
  # print('构建困难三元组ing...')
  for i in range(0,num_):
    classes = samples_Y[i]
    anchor = embed[i,:]
    ##################构建困难正样本##################
    #构建除anchor以外的pos样本
    pos_embed = embed[samples_Y == classes]
    pos_embed = np.delete(pos_embed,np.where(pos_embed == anchor)[0][0],axis=0)
    #reshape便于计算
    anchor = anchor.reshape((1,-1))
    #计算距离
    anchor = np.repeat(anchor, pos_embed.shape[0], axis=0)
    temp_dis=np.sum((anchor-pos_embed)*(anchor-pos_embed),axis=1)
    loc_p=temp_dis.argmax()#类内最大距离
    hard_pos=samples_X[loc_p,:,:,:]
    ##################构建困难负样本##################
    neg_embed = embed[samples_Y != classes]
    anchor = embed[i,:]
    anchor = anchor.reshape((1,-1))
    anchor = np.repeat(anchor, neg_embed.shape[0], axis=0)
    temp_dis=np.sum((anchor-neg_embed)*(anchor-neg_embed),axis=1)
    loc_n=temp_dis.argmin()#类间最小距离
    hard_neg=samples_X[loc_n,:,:,:]
    hard_triplet[i,0,:,:,:]=samples_X[i,:,:,:]
    hard_triplet[i,1,:,:,:]=hard_pos
    hard_triplet[i,2,:,:,:]=hard_neg
  return hard_triplet
#3.1 构建困难三元组嵌套
def process_triplet(data_X,label,model,num=num):
  #抽样
  sample_X,sample_Y=sampling(data_X,label,num=num)
  #获取困难三元组
  hard_tri=get_hard_triplet(sample_X,sample_Y,model)
  # print('困难三元组构建完成')
  return hard_tri

In [ ]:
def mixup(train_X,label,unlabeled,model,scale=scale):
  # Smodel.load_weights(weight_)
  #随机从无标签池中抽取样本
  temp_unlabeled=sampling_unlabel(unlabeled,scale)
  train_embed = model.predict(train_X)
  #开始mixup
  # count_all_label=[]
  plabel_all = []
  acc_all = []
  for i in range(len(temp_unlabeled)):
    data_enhanced=np.zeros((7,11,11,12))
    #减少增强次数
    # data_enhanced=np.zeros((6,11,11,7))
    data_enhanced[0,]=unlabeled[i,]
    #水平垂直对角翻转
    data_enhanced[1,]=np.flip(unlabeled[i,],axis=0)
    data_enhanced[2,]=np.flip(unlabeled[i,],axis=1)
    data_enhanced[3,]=np.flip(unlabeled[i,],axis=2)
    #增加噪声(轻噪，重噪)
    noise = np.random.normal(0.0, 0.01, size=unlabeled[i,].shape)
    data_enhanced[4,]=i+noise
    noise = np.random.normal(0.0, 0.05, size=unlabeled[i,].shape)
    data_enhanced[5,]=i+noise
    #随机旋转
    k = np.random.randint(4)
    data_enhanced[6,]=np.rot90(unlabeled[i,],k=k)
    #增强结果的embed
    enhance_embed=model.predict(data_enhanced)
    #计算度量获取伪标签
    pseudo_label=[]
    for j in range(len(enhance_embed)):
      #扩充向量维度，计算距离，找最小距离
      dis_list=[]
      #伪标签获取
      dist_mat = tf.reduce_sum(tf.square(tf.subtract(enhance_embed[j,].reshape((1,-1)),train_embed)),axis=-1)
      index = np.where(dist_mat == np.min(dist_mat))[0][0]
      # print(index)
      classes = label[index]    
      pseudo_label.append(classes)
    counts = np.bincount(pseudo_label)
    plabel= np.argmax(counts)
    acc=counts[plabel]/len(pseudo_label)
    plabel_all.append(plabel)
    acc_all.append(acc)

  #返回batch所有无标签样本和对应伪标签及概率
  plabel_all = np.asarray(plabel_all,dtype= int)
  acc_all = np.asarray(acc_all,dtype=float)
  return temp_unlabeled,plabel_all,acc_all
#获取pseudo triplet
def ranking(unlabeled_sample,p_label,p_prob,topk=topk):
  pseudo_triplet = []
  pseudo_label = []
  flag = 0 
  # ranksample = np.zeros((topk*))
  for i in range(1,8):
    classes = i
    # temp_label = p_label[p_label == classes]
    temp_prob = p_prob[(p_label == classes)& (p_prob > 0.57)]
    temp_sample = unlabeled_sample[(p_label == classes)& (p_prob > 0.57)]
    temp_rank = np.argsort(temp_prob,axis=0)
    length = len(temp_rank)
    if length >= topk:
      for j in range(0,topk):
        ranknum = np.where(temp_rank == j)[0][0]
        pseudo_triplet.append(temp_sample[ranknum,:])
        pseudo_label.append(i)
    else:
      for j in range(0,length):
        ranknum = np.where(temp_rank == j)[0][0]
        pseudo_triplet.append(temp_sample[ranknum,:])
        pseudo_label.append(i)
  final_pseudo_sample = np.array(pseudo_triplet)
  final_pseudo_label = np.asarray(pseudo_label,dtype=int)
  
  return final_pseudo_sample,final_pseudo_label


In [ ]:
#解决伪标签全为一类的问题
def judge(pseudo_label,prob,num):
  #判断随机伪样本是否至少除other类外都大于10num个
  count = 0
  for i in range(1,8):
    temp_prob = prob[pseudo_label == i]
    temp_prob = temp_prob[temp_prob > 0.57]
    if temp_prob.shape[0] >= num:
      count += 1
  if count >= 7:
    return True
  else:
    return False

In [ ]:
#三元组损失计算
def triplet_loss(triplet,alpha=alpha):
  """
  triplet=三元组
  """
  anchor,positive,negative=triplet[0],triplet[1],triplet[2]
  # print(anchor.shape)
  #第一步：计算"anchor" 与 "positive"之间编码的距离，这里需要使用axis=-1
  pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)#二范数，-1代表倒数第一个维度即64
  #第二步：计算"anchor" 与 "negative"之间编码的距离，这里需要使用axis=-1
  neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
  #第三步：减去之前的两个距离，然后加上alpha
  basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
  #通过取带零的最大值和对训练样本的求和来计算整个公式
  loss = tf.reduce_sum(tf.maximum(basic_loss,0))

  return loss

In [ ]:
def RPmodel(model,alpha=0.05,P_weight=0.2):
  """
  model:网络backbone
  alpha:margin参数
  P_weight:伪样本损失权重
  return:返回算好的损失
  """
  #真实样本损失
  y_pred_real=[]
  real_anchor = Input(inputshape)
  real_positive = Input(inputshape)
  real_negative = Input(inputshape)
  r_anchor_out = Model(real_anchor, model(real_anchor))
  r_positive_out = Model(real_positive, model(real_positive))
  r_negative_out = Model(real_negative, model(real_negative))
  y_pred_real.append(r_anchor_out.output)
  y_pred_real.append(r_positive_out.output)
  y_pred_real.append(r_negative_out.output)
  loss_real = Lambda(lambda x: triplet_loss(x, alpha), output_shape=[1])(y_pred_real)
  #伪样本损失
  y_pred_pseudo=[]
  pseudo_anchor = Input(inputshape)
  pseudo_positive = Input(inputshape)
  pseudo_negative = Input(inputshape)
  p_anchor_out = Model(pseudo_anchor, model(pseudo_anchor))
  p_positive_out = Model(pseudo_positive, model(pseudo_positive))
  p_negative_out = Model(pseudo_negative, model(pseudo_negative))
  y_pred_pseudo.append(p_anchor_out.output)
  y_pred_pseudo.append(p_positive_out.output)
  y_pred_pseudo.append(p_negative_out.output)
  loss_pseudo = Lambda(lambda x: triplet_loss(x, alpha), output_shape=[1])(y_pred_pseudo)
  
  tripletloss=loss_real+P_weight*loss_pseudo
  #输入正负样本的三元组，输出损失
  return Model(inputs = [real_anchor,real_positive,real_negative,pseudo_anchor,pseudo_positive,pseudo_negative],outputs=tripletloss)

In [ ]:
def train_model(model,train_X,train_Y,unlabeled):#待修改
  """
  model:backbone
  X_pos,X_neg:有标签正/负样本
  unlabeled：无标签样本池
  """
  num_times = 20 #训练num_times次，每次epoch轮
  P_weight1=0.0
  semimodel = RPmodel(model,alpha=0,P_weight = 0.1)
  optm = K.optimizers.Adam(lr=1e-4,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01)
  semimodel.compile(optimizer=optm,loss=lambda t,p:p)
  model.load_weights('/content/gdrive/MyDrive/CM/weight_folder/zhangbei_triplet_weight_alpha=0.h5',by_name=True)
  model_checkpoint = ModelCheckpoint('/content/gdrive/MyDrive/CM/weight_folder/zhangbei_triplet_weight_plus_pseudo_alpha=0.h5', monitor= 'loss', verbose=1, 
                                     save_best_only=True, save_weights_only=True,period=5) 
  batch_print_callback = LambdaCallback(on_epoch_end=lambda batch,logs: print('验证集精度：'+str(use_val(X_train,Y_train,X_val,Y_val,model))))
  early_stopping =EarlyStopping(monitor='loss', patience=5)
  for i in tqdm(range(num_times)):
    #真实样本困难三元组
    real_hard_tri = process_triplet(train_X,train_Y,model,num=5)
    print('真实样本困难三元组构建完成')
    # print('a')
    unlabeled_batch,class_label,class_prob=mixup(train_X,train_Y,unlabeled,model,scale=2000)
    # print(class_prob)
    while (judge(class_label,class_prob,num=5)==False):
      print("pseudo labeled failed, retry")
      unlabeled_batch,class_label,class_prob=mixup(train_X,train_Y,unlabeled,model,scale=2000)
      # judge(class_label,class_prob,num=10)
    print("success,continue")
    # pseudo_pos,pseudo_neg=ranking(unlabeled_batch,class_prob)
    pseudo_sample,pseudo_label = ranking(unlabeled_batch,class_label,class_prob,topk=5)
    # #真实样本池增加
    # X_positive = np.concatenate((X_positive,pseudo_pos),axis=0)
    # X_negative = np.concatenate((X_negative,pseudo_neg),axis=0)
    # print(X_positive.shape)
    # print(X_negative.shape)
    pseudo_hard_tri=get_hard_triplet(pseudo_sample,pseudo_label,model)
    print('伪样本困难三元组构建完成')
    # all_tri = np.zeros((real_hard_tri.shape[0],real_hard_tri.shape[1]*2,real_hard_tri.shape[2],real_hard_tri.shape[3],real_hard_tri.shape[4]))
    print(real_hard_tri.shape)
    print(pseudo_hard_tri.shape)
    all_tri = np.concatenate((real_hard_tri,pseudo_hard_tri),axis=1)
    print(all_tri.shape)
    y_label = np.zeros((len(all_tri),1))#用不着
    input_x = [
            all_tri[:,0,:,:,:],
            all_tri[:,1,:,:,:],
            all_tri[:,2,:,:,:],
            all_tri[:,3,:,:,:],
            all_tri[:,4,:,:,:],
            all_tri[:,5,:,:,:]
    ]
    semimodel.fit(x=input_x,y=y_label,batch_size=25,epochs=50,shuffle=True,callbacks=[model_checkpoint, early_stopping, batch_print_callback])
    P_weight1 += 0.025
    # #保存训练完成一次的权重，重新映射获取三元组
    # model.save('/content/gdrive/MyDrive/CM/recurrent_weight.h5')
    # positive_embed = embedding(X_pos,model)
    # negative_embed = embedding(X_neg,model)
    # model.save('/content/gdrive/MyDrive/CM/weight_folder/new_dataset_triplet_weight_alpha0.05.h5')

In [ ]:
def test_predict(train_X,train_Y,test_X,model):
  train_X_embed = model.predict(train_X)
  test_X_embed = model.predict(test_X)
  pred_class = []
  for i in range(0,test_X_embed.shape[0]):
    #计算到各个类别的距离
    dist_mat = tf.reduce_sum(tf.square(tf.subtract(test_X_embed[i],train_X_embed)),axis=-1)
    index = int(np.where(dist_mat == np.min(dist_mat))[0])
    classes = Y_train[index]
    pred_class.append(classes)
  return pred_class

In [ ]:
def use_val(train_X,train_Y,val_X,val_Y,model):
  pred = test_predict(train_X,train_Y,val_X,model)
  pred = to_categorical(pred)
  cm,oa,kappa = get_cm_oa_kappa(val_Y,pred)
  return oa

In [ ]:

# weight_='/content/gdrive/MyDrive/CM/weight_folder/zhangbei_triplet_weight_alpha=0.h5'

X_train = np.load('/content/gdrive/MyDrive/CM/roi_patches/zb_train_X_r=5.npy')
Y_train = np.load('/content/gdrive/MyDrive/CM/roi_patches/zb_train_Y_r=5.npy')
X_val = np.load('/content/gdrive/MyDrive/CM/roi_patches/zb_val_X_r=5.npy')
Y_val = np.load('/content/gdrive/MyDrive/CM/roi_patches/zb_val_Y_r=5.npy')
X_test = np.load('/content/gdrive/MyDrive/CM/roi_patches/zb_test_X_r=5.npy')
Y_test = np.load('/content/gdrive/MyDrive/CM/roi_patches/zb_test_Y_r=5.npy')
unlabeled = np.load('/content/gdrive/MyDrive/CM/roi_patches/zhangbei_unlabeled_X.npy')

In [ ]:
Smodel=feature_extraction_CNN_tri(inputshape)
# Smodel.load_weights(weight_,by_name=True)

输出向量维度：(None, 256)


In [ ]:
num_times = 20 #训练num_times次，每次epoch轮
P_weight1=0.0
# semimodel = RPmodel(Smodel,alpha=0.2,P_weight = P_weight1)

(None, 256)
(None, 256)


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, cohen_kappa_score
def get_OA(cm):
    
    total = cm.sum()
    
    diag_arr = np.diagonal(cm, offset=0)
    
    correct = diag_arr.sum()
    
    OA = (correct/total) * 100
    
    return OA

def get_cm_oa_kappa(Y_test, Y_pred,threshold=0.8):
    # pred_y=[]
    # print(threshold)
    # for i in Y_pred:
    #   # print(i[1])
   
    #   if i[1]>=threshold:
    #     temp_y=Y_pred[i]
    #     pred_y.append(temp_y)
    #   else:
    #     temp_y=0
    #     pred_y.append(temp_y)
    # # print(Y_test,Y_pred)
    # # print(pred_y)
    # cm = confusion_matrix(pred_y, Y_test)
    # oa = get_OA(cm)
    # kappa = cohen_kappa_score(pred_y, Y_test)

    ###############################
    Y_pred = np.argmax(Y_pred, axis=1)
    
    Y_test = Y_test.tolist()
    Y_pred = Y_pred.tolist()
    # print(Y_pred)
    # print('\n')
    # print(Y_test)
  
    cm = confusion_matrix(Y_pred, Y_test)
    
    oa = get_OA(cm)
    
    kappa = cohen_kappa_score(Y_pred, Y_test)
    
    # print(cm)
    
    return cm, oa, kappa

In [ ]:
train_model(Smodel,X_train,Y_train,unlabeled)

In [ ]:
#############测试模型
Tmodel=feature_extraction_CNN_tri(inputshape)
weight_ = '/content/gdrive/MyDrive/CM/weight_folder/zhangbei_triplet_weight_plus_pseudo_alpha=0.h5'
Tmodel.load_weights(weight_,by_name=True)

输出向量维度：(None, 256)


In [ ]:
pred = test_predict(X_train,Y_train,X_test,Tmodel)

In [ ]:
pred1 = to_categorical(pred)

In [ ]:
cm, oa, kappa = get_cm_oa_kappa(Y_test, pred1)
print('acc: {:.2f}%  Kappa: {:.4f}'.format(oa,kappa))
# print('acc: {:2f}%'.format(oa))
print(cm)

acc: 82.92%  Kappa: 0.7961
[[73  5  1  0  0  0  0]
 [12 85  1  2  0  0  0]
 [ 5  0 88  0  3  7  0]
 [ 0  0  0 47  0  0  3]
 [ 0  0  0  0 55 10  1]
 [ 0  0  0  3 32 73  0]
 [ 0  0  0  4  0  0 11]]


In [ ]:
for i in range(50):
  Tmodel=feature_extraction_CNN_tri(inputshape)
  weight_ = '/content/gdrive/MyDrive/CM/weight_folder/zhangbei_triplet_weight_alpha=0.h5'
  Tmodel.load_weights(weight_,by_name=True)
  pred = test_predict(X_train,Y_train,X_test,Tmodel)
  pred1 = to_categorical(pred)
  cm, oa, kappa = get_cm_oa_kappa(Y_test, pred1)
  print('acc: {:.2f}%  Kappa: {:.4f}'.format(oa,kappa))
  # print('acc: {:2f}%'.format(oa))
  print(cm)

输出向量维度：(None, 256)
acc: 82.53%  Kappa: 0.7915
[[67  9  0  0  0  0  0]
 [18 81  0  2  0  0  0]
 [ 5  0 89  0  7 10  0]
 [ 0  0  1 52  0  0  1]
 [ 0  0  0  0 59 10  2]
 [ 0  0  0  1 24 70  0]
 [ 0  0  0  1  0  0 12]]
输出向量维度：(None, 256)
acc: 81.77%  Kappa: 0.7822
[[71  7  0  0  0  0  0]
 [17 83  3  2  0  0  0]
 [ 2  0 87  0  8  7  0]
 [ 0  0  0 48  0  0  0]
 [ 0  0  0  0 56 13  4]
 [ 0  0  0  3 26 70  0]
 [ 0  0  0  3  0  0 11]]
输出向量维度：(None, 256)
acc: 82.15%  Kappa: 0.7866
[[68  5  0  0  0  0  0]
 [18 85  2  5  0  0  0]
 [ 4  0 88  0  7 10  0]
 [ 0  0  0 49  0  0  4]
 [ 0  0  0  0 60  9  4]
 [ 0  0  0  0 23 71  0]
 [ 0  0  0  2  0  0  7]]
输出向量维度：(None, 256)
acc: 79.85%  Kappa: 0.7592
[[61 10  0  1  0  0  0]
 [27 80  3  1  0  0  0]
 [ 2  0 87  0  9  9  0]
 [ 0  0  0 52  0  0  0]
 [ 0  0  0  0 49  5  4]
 [ 0  0  0  2 32 76  0]
 [ 0  0  0  0  0  0 11]]
输出向量维度：(None, 256)
acc: 82.73%  Kappa: 0.7936
[[75  8  0  0  0  0  0]
 [12 82  1  2  0  0  0]
 [ 3  0 89  0  5  8  0]
 [ 0  0  0 50  0  0  2